# More Advanced Functional Models - Part 2

Using the site <https://www.tensorflow.org/guide/keras/functional?hl=en> as a guide to practice coding up more complex models.

## Complex Graph Topologies

### Multiple inputs and outputs

Setting up a system for ranking customer issue tickets by priority and routing them to the correct department.  

The model will have 3 inputs:
* title of the ticket (text input)
* test of the ticket (text input)
* tags (categorical input)

This model will have 2 outputs:
* the priority score of 0 - 1 (scalar sigmoid outputs)
* department that should handle the ticket (softmax output over the set of departments)

In [ ]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

In [ ]:
# Define the number of tags, words, and departments
num_tags = 12
num_words = 10000
num_department = 4

In [ ]:
title_input = tf.keras.Input(shape=(None,), name="title")  # variable length of sequence
body_input = tf.keras.Input(shape=(None,), name="body") # again, sequence text for body
tags_input = tf.keras.Input(shape=(num_tags,),name="tags") # category of text

# create embeddings
title_features = tf.keras.layers.Embedding(num_words, 64)(title_input)
body_features = tf.keras.layers.Embedding(num_words, 64)(body_input)

# reduce sequence of embedded words into a single 128-dim vector
title_features = tf.keras.layers.LSTM(128)(title_features)
# reduce sequence of embedded words into 32 dim vectors
body_features = tf.keras.layers.LSTM(128)(body_features)

# merge all available features into a single large vector
x = tf.keras.layers.concatenate([title_features, body_features, tags_input])

# define outputs for priority
priority_pred = tf.keras.layers.Dense(1, name="priority")(x)
# define outputs for department
department_pred = tf.keras.layers.Dense(num_department, name="department")(x)

model = tf.keras.Model(
    inputs=[title_input, body_input, tags_input],
    outputs=[priority_pred, department_pred]
)

![plot model](./model_plot/output_52c4dc6fd93e_0.png)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-3),
    loss=[
        tf.keras.losses.BinaryCrossentropy(from_logits=True),
        tf.keras.losses.CategoricalCrossentropy(from_logits=True)   
    ],
    loss_weights=[1.0,0.2]
)

In [ ]:
print(model.to_json())

### Creating dummy data

In [ ]:
# Dummy data
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280,100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype("float32")

# dummy target
priority_target = np.random.random(size=(1280,1))
dept_targets = np.random.randint(2, size=(1280, num_department))

In [ ]:
model.fit(
    {
        "title": title_data, 
        "body": body_data,
        "tags": tags_data
    },
    {
        "priority": priority_target,
        "department": dept_targets
    },
    epochs=2,
    batch_size=32,
)